In [1]:
import numpy as np
import pandas as pd

from pandas import read_csv

from matplotlib import pyplot
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import Image
from IPython.core.display import HTML



import sklearn
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn import decomposition
from sklearn.decomposition import pca
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB

import scipy

from scipy.stats.stats import spearmanr
from scipy.stats import chi2_contingency

%matplotlib inline

In [2]:
df = pd.read_csv('bank-full.csv', delimiter = ';')
df.head()

df_2 = pd.DataFrame()

#Label encoder instantiation
labelencoder = LabelEncoder()
df_2['y'] = labelencoder.fit_transform(df['y'])

# Get dummies variables
data_dummies = pd.get_dummies(df)
merged = pd.concat([df_2, data_dummies], axis = "columns")
merged.head()

,y,age,balance,day,duration,campaign,pdays,previous,job_admin.,job_blue-collar,...,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown,y_no,y_yes
0,0,58,2143,5,261,1,-1,0,0,0,...,1,0,0,0,0,0,0,1,1,0
1,0,44,29,5,151,1,-1,0,0,0,...,1,0,0,0,0,0,0,1,1,0
2,0,33,2,5,76,1,-1,0,0,0,...,1,0,0,0,0,0,0,1,1,0
3,0,47,1506,5,92,1,-1,0,0,1,...,1,0,0,0,0,0,0,1,1,0
4,0,33,1,5,198,1,-1,0,0,0,...,1,0,0,0,0,0,0,1,1,0


In [3]:
#df = pd.read_csv('bank-full.csv', delimiter = ';')
df_1 = df[['age','marital','education','job','balance','default','housing','loan','duration','campaign','pdays','previous']]
df_2 = pd.DataFrame()

#Label encoder instantiation
labelencoder = LabelEncoder()
df_2['y'] = labelencoder.fit_transform(df['y'])

# Get dummies variables
data_dummies = pd.get_dummies(df_1)
merged = pd.concat([df_2, data_dummies], axis = "columns")

#Avoid dummy trap
X = merged.drop(['marital_divorced','education_primary','job_blue-collar','default_no','housing_no','loan_no'], axis="columns")


# Separating out the labels and features
y = X.iloc[:,0].values
X = X.iloc[:,1:26].values
X

array([[  58, 2143,  261, ...,    0,    1,    0],
       [  44,   29,  151, ...,    0,    1,    0],
       [  33,    2,   76, ...,    0,    1,    1],
       ..., 
       [  72, 5715, 1127, ...,    0,    0,    0],
       [  57,  668,  508, ...,    0,    0,    0],
       [  37, 2971,  361, ...,    0,    0,    0]])

In [4]:
# Resampling the minority to have the same number of sample with majority with synthetic resampling with KNN
from imblearn.over_sampling import SMOTE

smote = SMOTE(ratio='minority')
X, y = smote.fit_sample(X, y)

# Standardizing the features
#X = StandardScaler().fit_transform(X)
#from sklearn.decomposition import PCA
#pca = PCA(n_components=8)
#X = pca.fit_transform(X)

#pca.explained_variance_ratio_

In [5]:
y = y.ravel()
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 1)


In [31]:
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, VotingClassifier
adb = AdaBoostClassifier(DecisionTreeClassifier(), n_estimators = 5, learning_rate = 1)
adb.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
          learning_rate=1, n_estimators=5, random_state=None)

In [48]:
#Bagging classifier - Gradient

bg = BaggingClassifier(DecisionTreeClassifier(), max_samples = 0.5, max_features = 1.0, n_estimators = 20)
bg.fit(X_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=0.5, n_estimators=20, n_jobs=None, oob_score=False,
         random_state=None, verbose=0, warm_start=False)

In [49]:
# Random forest- Ensemble of Decision Trees
rf = RandomForestClassifier(n_estimators=10)
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [95]:
vvv = VotingClassifier( estimators = [('rf', rf),('bg', bg),('adb', adb),('rff', rff),('lr', lr),('nb', nb)], voting = 'hard')
vvv.fit(X_train, y_train)

VotingClassifier(estimators=[('rf', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_we...warm_start=False)), ('nb', BernoulliNB(alpha=1.0, binarize=True, class_prior=None, fit_prior=True))],
         flatten_transform=None, n_jobs=None, voting='hard', weights=None)

In [6]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)

/home/seggs/anaconda/lib/python2.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [81]:
dt = DecisionTreeClassifier(criterion="entropy", random_state= 100, max_depth= 3, min_samples_leaf=5)
dt.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=100,
            splitter='best')

In [51]:
rff = RandomForestClassifier(n_jobs=2, random_state=0)

rff.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=2,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [82]:
nb = BernoulliNB(binarize= True)
nb.fit(X_train, y_train)

BernoulliNB(alpha=1.0, binarize=True, class_prior=None, fit_prior=True)

In [66]:
model = VotingClassifier( estimators = [('rff', rff),('lr', lr)], voting = 'hard')
model.fit(X_train, y_train)

VotingClassifier(estimators=[('rff', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_w...penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))],
         flatten_transform=None, n_jobs=None, voting='hard', weights=None)

In [7]:
from sklearn.metrics import accuracy_score
y_pred = lr.predict(X_test)
print(accuracy_score(y_test, y_pred))
from sklearn.metrics import confusion_matrix
cnf_matrix = confusion_matrix(y_test, y_pred)
cnf_matrix

0.886154424197


array([[7082,  903],
       [ 915, 7069]])

In [8]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

print("Accuracy:",accuracy_score(y_test, y_pred))
print("Precision:",precision_score(y_test, y_pred))
print("Recall:",recall_score(y_test, y_pred))

('Accuracy:', 0.88615442419688151)
('Precision:', 0.8867285499247366)
('Recall:', 0.88539579158316628)


In [9]:
from sklearn.metrics import classification_report
target_names = ['0', '1']
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

           0       0.89      0.89      0.89      7985
           1       0.89      0.89      0.89      7984

   micro avg       0.89      0.89      0.89     15969
   macro avg       0.89      0.89      0.89     15969
weighted avg       0.89      0.89      0.89     15969



In [99]:
from sklearn.model_selection import cross_val_score
print(cross_val_score(model, X, y, cv=10)) 

[ 0.58014025  0.84585525  0.92910822  0.93687375  0.94463928  0.94388778
  0.92773046  0.93036072  0.93461924  0.8613477 ]


In [62]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
y_pred_proba = model.predict_proba(X_test)[::,1]
fpr, tpr, _ = roc_curve(y_test,  y_pred_proba)
auc = roc_auc_score(y_test, y_pred_proba)
plt.plot(fpr,tpr,label="data , auc="+str(auc))
plt.legend(loc=4)
plt.show()

AttributeError: predict_proba is not available when voting='hard'